# Using the MANN Package to convert and prune an existing TensorFlow model

In this notebook, we utilize the MANN package on an existing TensorFlow model to convert existing layers to MANN layers and then prune the model using RSN2.

In [1]:
# Load the MANN package and TensorFlow
import tensorflow as tf
import mann

In [2]:
# Load the data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train/255
x_test = x_test/255

# Load the model to be used
vgg16 = tf.keras.applications.VGG16(
    include_top = False,             # Don't include the top layers
    weights = 'imagenet',            # Load the imagenet weights
    input_shape = x_train.shape[1:]  # Input shape is the shape of the images
)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-03 08:01:16.867707: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-03 08:01:16.867812: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Build the model using VGG16 and a few layers on top of it
model = tf.keras.models.Sequential()
model.add(vgg16)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(512, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation = 'softmax'))

# Compile the model
model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Present model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 15,507,786
Trainable params: 15,507,786
Non-trainable params: 0
____________________________________________

In [4]:
# Use the add_layer_masks function to add masking layers to the model
converted_model = mann.utils.add_layer_masks(model)

# Compile the model
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Mask the model using magnitude as the metric
converted_model = mann.utils.mask_model(
    converted_model,
    40,
    method = 'magnitude'
)

# Recompile the model for the weights to take effect
converted_model.compile(
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy'],
    optimizer = 'adam'
)

# Present the model summary
converted_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         29429376  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
masked_dense (MaskedDense)   (None, 512)               525312    
_________________________________________________________________
masked_dense_1 (MaskedDense) (None, 512)               525312    
_________________________________________________________________
masked_dense_2 (MaskedDense) (None, 512)               525312    
_________________________________________________________________
masked_dense_3 (MaskedDense) (None, 10)                10260     
Total params: 31,015,572
Trainable params: 15,507,786
Non-trainable params: 15,507,786
___________________________________

In [5]:
# Create the sparsification callback object
callback = mann.utils.ActiveSparsification(
    0.75,
    starting_sparsification = 40,
    sparsification_rate = 5
)

# Fit the model
model.fit(
    x_train,
    y_train,
    epochs = 1000,
    callbacks = [callback],
    validation_split = 0.2,
    batch_size = 256
)

2022-03-03 08:01:18.327110: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-03 08:01:18.327285: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/1000


2022-03-03 08:01:18.529887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - ETA: 0s - loss: 1.9852 - accuracy: 0.2017

2022-03-03 08:02:10.787439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


157/157 [==============================] - 56s 354ms/step - loss: 1.9852 - accuracy: 0.2017 - val_loss: 1.7798 - val_accuracy: 0.2741
Performance measure set to val_accuracy
Model performance has not reached pruning threshold for 1 epoch(s)
Epoch 2/1000
157/157 [==============================] - 54s 346ms/step - loss: 1.7332 - accuracy: 0.2871 - val_loss: 1.8261 - val_accuracy: 0.2843
Model performance has not reached pruning threshold for 2 epoch(s)
Epoch 3/1000
157/157 [==============================] - 54s 345ms/step - loss: 1.4679 - accuracy: 0.4106 - val_loss: 1.4511 - val_accuracy: 0.4686
Model performance has not reached pruning threshold for 3 epoch(s)
Epoch 4/1000
157/157 [==============================] - 55s 350ms/step - loss: 1.1497 - accuracy: 0.5736 - val_loss: 0.9907 - val_accuracy: 0.6472
Model performance has not reached pruning threshold for 4 epoch(s)
Epoch 5/1000
157/157 [==============================] - 55s 350ms/step - loss: 0.8669 - accuracy: 0.6986 - val_loss: 

In [6]:
# Convert the model back
model = mann.utils.remove_layer_masks(model)

# Present the model
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 15,507,786
Trainable params: 0
Non-trainable params: 15,507,786
____________________________________________

In [10]:
# Get the predictions on test data
preds = model.predict(x_test).argmax(axis = 1)

# Print the accuracy
print(f'Model Accuracy: {(preds.flatten() == y_test.flatten()).sum().astype(int)/y_test.flatten().shape[0]}')

# Save the model
model.save('cifar_vgg16.h5')

Model Accuracy: 0.7962
